In [7]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", "neo4jneo4j"))

In [15]:
query = """
CALL gds.graph.project(
    'got1',
    'Person',
    {
        INTERACT: { properties : 'weight'}
    },
    {nodeProperties:'seed'}
    )
YIELD
  graphName AS graph,
  relationshipProjection AS knowsProjection,
  nodeCount AS nodes,
  relationshipCount AS rels
"""
graph.run(query)

graph,knowsProjection,nodes,rels
got1,"{INTERACT: {orientation: 'NATURAL', indexInverse: false, aggregation: 'DEFAULT', type: 'INTERACT', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",399,2433


In [16]:
query = """
CALL gds.pageRank.stream('got1',  {maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS page, score
ORDER BY score DESC
LIMIT 5
"""

graph.run(query)


page,score
TYRION,7.2829717554208235
VARYS,5.674118882216431
WALDER,4.602201092362613


In [17]:
query = """
CALL gds.louvain.write.estimate('got1', { writeProperty: 'community' })
YIELD nodeCount, relationshipCount, bytesMin, bytesMax, requiredMemory
"""
graph.run(query)

nodeCount,relationshipCount,bytesMin,bytesMax,requiredMemory
399,2433,30881,634216,[30 KiB ... 619 KiB]


In [20]:
graph.run("CALL gds.louvain.stats('got1')YIELD communityCount")


communityCount
23


In [13]:
query = """
CALL gds.louvain.stream('got1',{ seedProperty: 'seed' })
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).label AS label, communityId
ORDER BY label 
LIMIT 5
"""

graph.run(query)

label,communityId
Addam,151
Adrack Humble,94
Aegon,51


In [14]:
query = """
CALL gds.graph.drop('got1') YIELD graphName;
"""
graph.run(query)

graphName
got1
